# LLM recreation (exploratory NB)

## Goals
1. Transformer
  * MVP 4/14
  * Make code very clean 4/16
2. RLHF
  * MVP 4/21
3. Enhance! 4/28
  * Open-ended enhancement time, TBD
  * Fine-tune for Paul GPT?
4. UI? Blog? LI post? 5/5

## Resources:
* Start with Andreu Karpathy's [Youtube Transformer Walkthrough](https://www.youtube.com/watch?v=kCc8FmEb1nY) (Sid's recommendation)
  * Progress: 0:00
* See anthropic email.
  * "Decoder only"
  * PyTorch
  * "Start with just a free GPU from Google colab, then scale up to an 8x A100 instance"

* [AGISF career guide](https://www.agisafetyfundamentals.com/alignment-careers-guide) --> How to pursue research contributor --> Do some paper replications.
  * [GPT2 paper](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
  * [Anthropic blog post](https://www.alignmentforum.org/posts/YDF7XhMThhNfHfim9/ai-safety-needs-great-engineers) on how to become an AIS eng.
    * (TLDR: can you pull request to a major ML library?)
* Our AGISF miro board: https://miro.com/app/board/uXjVPo1UFPU=/ 
* Advice for paper replication: https://forum.effectivealtruism.org/posts/fRjj6nm9xbW4kFcTZ/advice-on-pursuing-technical-ai-safety-research#2_1__Advice_on_paper_replication
* Can rent more GPUs here: https://jarvislabs.ai/


## TODO:
### Read papers/posts (roughly high-to-low priority)
* [Attention is all you need](https://arxiv.org/abs/1706.03762) (Transformers)
* [Deep Residual Learning...](https://arxiv.org/abs/1512.03385) (Skip Connections)
* [Dropout: A simple way...](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)

## Next Steps/Fixes
* git
* Print losses averaged over batches.
* Convert things to nn.Sequential
* Multi-head into a 4th dimension??
* GPU enabled.
* Classes to script

## Ideas:

### UI (personal site)
* Side-by side with GPT3.5 API?
* Deploy to Minecraft server? Is there an easy text2speech and speech2text api I could use?


## Misc Notes
### Andreu Karpathy's [Youtube Transformer Walkthrough](https://www.youtube.com/watch?v=kCc8FmEb1nY)

**Progress: 1:38:00**

GPT = Generatively Pretrained Transformer

Character-level transformer model. Smaller dataset: Tiny shakespeare (1MB)

At 18:00, he mentions that transformer can only see truncated input up to block size. But does transformer store more knowledge than this? I bet yet.

At 22:00, if block size is 8, then we have 8 similar training examples, where starting word is always the same. Is this a good idea, or better to just have purely random, scattered starting points from all over the data set? 
* I understand desire to have varying length, but length and starting point can just both be randomly picked
* Does this make the data more compact to feed into model?

**Bigram Language Model**


1:15:00:
"Decoder" has the lower triangular masking, sort of autoregressive, so that words can only see past words. 

"Encoder" removes this constraint, and we can look everywhere at once. This is fine for some cases, e.g., sentiment analysis.

"Attention" supports arbitrary directed graph, and can be used in either. Doesn't even have to be a linear, chronologic list!

"self attention" means that q, k, v is all a function of the same source "inputs". But this can be generalized!

"cross attention" can have keys/values generated from a "context", or corpus of data, and the query comes from the prompt. This is fine!




In [1]:
!ls

LLM_exploratory.ipynb


In [3]:
# from google.colab import data_table
import torch


In [2]:

# data_table.enable_dataframe_formatter()

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-18 00:07:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.76MB/s    in 0.2s    

2023-04-18 00:07:53 (4.76 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

In [ ]:
text

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [ ]:
len(text)

1115394

In [4]:
def get_encoder_decoder_size(text):
  vocab = sorted(list(set(text)))
  # print(''.join(vocab))
  vocab_size = len(vocab)
  # vocab_size
  str2code = {c: i for i, c in enumerate(vocab)}
  code2str = {i: c for i, c in enumerate(vocab)}

  encode = lambda text: [str2code[c] for c in text]
  def decode(code_list): 
    if type(code_list) == torch.Tensor:
      return decode(code_list.tolist())
    if type(code_list) == int:
      return code2str[code_list]
    return ''.join([code2str[i] for i in code_list]) 

  return encode, decode, vocab_size


In [ ]:
encode, decode, vocab_size = get_encoder_decoder_size(text)

In [ ]:
encode("Hhhhhowdy ho!")

[20, 46, 46, 46, 46, 53, 61, 42, 63, 1, 46, 53, 2]

In [ ]:
decode([20, 46, 46, 46, 46, 53, 61, 42, 63, 1, 46, 53, 2])

'Hhhhhowdy ho!'

In [5]:
def text_to_tv_tensors(text, encode):
  data = torch.tensor(encode(text))
  print(data.shape, data.dtype)
  print(data[:100])

  val_cutoff = int(len(data)*0.9)
  train = data[:val_cutoff]
  validate = data[val_cutoff:]
  len(train), len(validate)

  return train, validate

In [ ]:
train, validate = text_to_tv_tensors(text, encode)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
block_size = 8 # this is small!
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
decode(train[:block_size+1])

'First Cit'

In [6]:
BLOCK_SIZE = 8
BATCH_SIZE = 4
torch.manual_seed(42)

def get_batch(data):

  n = len(data)
  idxs = torch.randint(n - BLOCK_SIZE, (BATCH_SIZE,))

  xs = [data[i:i+BLOCK_SIZE] for i in idxs]
  ys = [data[i+1:i+BLOCK_SIZE+1] for i in idxs]

  return torch.stack(xs), torch.stack(ys)


In [ ]:

train_batch_x, train_batch_y = get_batch(train)
validate_batch_x, validate_batch_y = get_batch(validate)
  

In [ ]:
print(train_batch_x.shape, train_batch_y.shape)
print(validate_batch_x.shape, validate_batch_y.shape)

torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([4, 8]) torch.Size([4, 8])


In [ ]:
print([decode(x) for x in train_batch_x])
print([decode(x) for x in train_batch_y])

['s his lo', ' there c', 'ENCE:\nTo', 'st, with']
[' his lov', 'there ca', 'NCE:\nTo ', 't, with\n']


In [ ]:
for b in range(BATCH_SIZE):
  for i in range(BLOCK_SIZE):
    # print()
    print(f"{decode(train_batch_x[b,:i])} --> {decode(int(train_batch_x[b,i]))}")

 --> s
s -->  
s  --> h
s h --> i
s hi --> s
s his -->  
s his  --> l
s his l --> o
 -->  
  --> t
 t --> h
 th --> e
 the --> r
 ther --> e
 there -->  
 there  --> c
 --> E
E --> N
EN --> C
ENC --> E
ENCE --> :
ENCE: --> 

ENCE:
 --> T
ENCE:
T --> o
 --> s
s --> t
st --> ,
st, -->  
st,  --> w
st, w --> i
st, wi --> t
st, wit --> h


In [ ]:
a =1 
b = [1,2,3]

In [ ]:
type(b) == int

False

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(43)

class BigramLanguageModule(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, vocab_size)

  def forward(self, inputs, targets=None):
    logits = self.embedding(inputs)
    B, T, C = logits.shape
    # E.g., 4 x 8 x 65 = examples/batch x chars per example (time) x vocab_size

    # It may seem odd that these targets are indices, where logits are scores
    # Basically, pytorch can handle EITHER indices as targets, or the actual OHE/probability ground truths. 
    #   could try target smoothing by OHE to 0.01 and 0.99 instead of 0/1?
    
    if targets is None:
      loss = None
    else:
      loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

    return logits, loss

  def generate(self, inputs, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(inputs) #B, T, C
      logits_last = logits[:,-1,:] #end of string only
      probs_last = F.softmax(logits_last, dim=1)
      next_char = torch.multinomial(probs_last, num_samples=1)

      inputs = torch.cat((inputs, next_char), dim=1) #B, T+1

    return inputs



In [ ]:
m = BigramLanguageModule(vocab_size)

In [ ]:
m(train_batch_x)[1]

In [ ]:
logits, loss = m.forward(train_batch_x, train_batch_y)
logits.shape, loss

(torch.Size([4, 8, 65]), tensor(4.7655, grad_fn=<NllLossBackward0>))

In [ ]:
logits[:,-1,:].shape

torch.Size([4, 65])

In [ ]:
B, T, C = logits.shape

In [ ]:

train_batch_x.shape

torch.Size([4, 8])

In [ ]:
train_batch_y

tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])

In [ ]:
# train_batch_x
# gen_train_batch_x = m.generate(train_batch_x, 10)

# for b in range(BATCH_SIZE):
#   print(f"{decode(train_batch_x[b,])} --> {decode(gen_train_batch_x[b,])}")

In [ ]:
m.parameters

<bound method Module.parameters of BigramLanguageModule(
  (embedding): Embedding(65, 65)
)>

In [ ]:
# optimizer = torch.optim.SGD(m.parameters(), lr=1e-3). #WWWWAY WORSE!
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
BATCH_SIZE = 32

for epoch in range(10000):
  tx, ty = get_batch(train)

  _, loss = m(tx, ty)
  optimizer.zero_grad(set_to_none=True)
  if epoch%1000 == 0:
    print(epoch, loss)
  loss.backward()
  optimizer.step()

0 tensor(2.9238, grad_fn=<NllLossBackward0>)
1000 tensor(2.6818, grad_fn=<NllLossBackward0>)
2000 tensor(2.5228, grad_fn=<NllLossBackward0>)
3000 tensor(2.4703, grad_fn=<NllLossBackward0>)
4000 tensor(2.4486, grad_fn=<NllLossBackward0>)
5000 tensor(2.5245, grad_fn=<NllLossBackward0>)
6000 tensor(2.4542, grad_fn=<NllLossBackward0>)
7000 tensor(2.5766, grad_fn=<NllLossBackward0>)
8000 tensor(2.3631, grad_fn=<NllLossBackward0>)
9000 tensor(2.4860, grad_fn=<NllLossBackward0>)


In [ ]:
seed_raw = "To be or"
seed = torch.tensor(encode(seed_raw)).view(1,-1)
print(decode(m.generate(seed, 500)[0,]))

NameError: ignored

My first language model output! Prompt =  "To be or..." (cross entropy ~2.5)

To be orory ay, hifl?\nPrke the the, s n wins\nESimavere mewapart sucind s lousith,\nS:\n\narr beise lfelesozer, 

Another example (500 chars):

To be oredd?
Isthiss thashorin wayof.
ONBcrrd fe the d m hu sam kser: t tllliceandHA:
PUKINUNan,
AToforicqukil dececo thy:
Whar wieil moulomer winou Fay!
Tjoourandoure y?
Fot bes-the; k alin, hisarot ghatened:
ALI tof y ht hthees tar; ethen ld byon tedir ds ten isp thed hthinch g mes tot terllinthalle bertthivimowy patoyonggianonneth arordwshofo t tom t alfidiea
Thingame omeld I m, ceaviehin:
Be aik! s, troneaveseits;
In, o bor.
Ayey; ats'dex's th iseener; hhallay bldinedasis t ine se hand O:
I sinoldou

Compared to untrained:

To be oron,\nJzt'!.&bTNZmXG\$RogNz xjLEMqWIB&eVL-pPZswMQCK:EHb3CMsomWOkZ3Eo\nJHXtChyoMXGeS:tC$OJrMYkZgNm;BT;OU?

**Definite improvement!**

In [ ]:
m2 = BigramLanguageModule(vocab_size)
print(decode(m2.generate(seed, 100)[0,]))

To be or?:DYeEyP
lkTLpcCqzrhGY!L;BEyxHflvc3VuUzfS;x:jDeeClACdN
tCVdTO.qvN-l;pOjBG KZF AbB
osjmj;YX;U
ibI!I$F


In [ ]:
seed

tensor([[32, 53,  1, 40, 43,  1, 53, 56]])

Stack the last 8 chars?

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(43)

class EightGramLanguageModule(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 32)
    self.lin1 = nn.Linear(32, 16)
    self.lin2 = nn.Linear(128, vocab_size)

  def forward(self, inputs, targets=None):
    # print()
    # print(inputs.shape)
    # print(targets.shape)
    embedded_input = self.embedding(inputs)
    B, T, C = embedded_input.shape
    # E.g., 4 x 8 x 32 = examples/batch x chars per example (time) x vocab_size

    # print(embedded_input.shape)

    fc1 = self.lin1(embedded_input)
    # 4 x 8 x 16

    fc1_flat = fc1.view(B, -1) #4, 8*16= 128
    # print(fc1_flat.shape)
    fc2 = self.lin2(fc1_flat) # 4, 65

    logits = fc2

    # It may seem odd that these targets are indices, where logits are scores
    # Basically, pytorch can handle EITHER indices as targets, or the actual OHE/probability ground truths. 
    #   could try target smoothing by OHE to 0.01 and 0.99 instead of 0/1?
    
    if targets is None:
      loss = None
    else:
      # loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
      # print("loss stuff")
      # print(logits.shape)
      # print(targets.shape)
      loss = F.cross_entropy(logits, targets[:,-1])
      # loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

    return logits, loss

  def generate(self, inputs, max_new_tokens):
    for _ in range(max_new_tokens):
      # print(inputs.shape)
      logits, _ = self(inputs[:,-8:]) #B, T, C
      # print(logits.shape)
      # logits_last = logits[:,-1] #end of string only
      probs_last = F.softmax(logits, dim=1)
      next_char = torch.multinomial(probs_last, num_samples=1)

      inputs = torch.cat((inputs, next_char), dim=1) #B, T+1

    return inputs



In [ ]:
m8 = EightGramLanguageModule(vocab_size)
# print(decode(m8.generate(seed, 10)[0,]))
tx, ty = get_batch(train)
_ = m8(tx)
print(decode(m8.generate(seed, 10)[0,]))

RuntimeError: ignored

In [ ]:

print(decode(m8.generate(seed, 100)[0,]))

NameError: ignored

In [ ]:
# optimizer = torch.optim.AdamW(m8.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(m8.parameters(), lr=1e-4)
BATCH_SIZE = 128

for epoch in range(10000):
  tx, ty = get_batch(train)

  _, loss = m8(tx, ty)
  optimizer.zero_grad(set_to_none=True)
  if epoch%1000 == 0:
    print(epoch, loss)
  loss.backward()
  optimizer.step()

RuntimeError: ignored

In [ ]:

print(decode(m8.generate(seed, 500)[0,]))

To be orimla d liad has
Rokingests.

DUNI Tori hatl your'ss pristuls:
And ms, ooreno shald'd caneen ta diksun!
There or med
brtane
Pat In:
Fit trot dire ofuth,
CLATE'D Whis big bur apnteessetr.

SARIGBEMPB:
I th ne siflusto chy ptoman ind tongle bore be you forsok', wid ryt ur lout to thald no tor igste thamr, an illerof on'm.

ThO tord-?
Whot of whit sndo chas thang atleday me.

MIOTIO:
Ye winkzul ofe tho cevouk ro for mperot, themy pale.

face bat
AndGNUUS:
Tnot ant falis's tith agots of ther hand Ori


previous 8-letter language model results (cross entropy ~2):

To be orimla d liad has
Rokingests.

DUNI Tori hatl your'ss pristuls:
And ms, ooreno shald'd caneen ta diksun!
There or med
brtane
Pat In:
Fit trot dire ofuth,
CLATE'D Whis big bur apnteessetr.

SARIGBEMPB:
I th ne siflusto chy ptoman ind tongle bore be you forsok', wid ryt ur lout to thald no tor igste thamr, an illerof on'm.

ThO tord-?
Whot of whit sndo chas thang atleday me.

MIOTIO:
Ye winkzul ofe tho cevouk ro for mperot, themy pale.

face bat
AndGNUUS:
Tnot ant falis's tith agots of ther hand Ori


## Next up:
Wiki data, GPUs!


In [ ]:
torch.cuda.is_available()

False

In [ ]:
!ls sample_data


anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
!pwd

/content


In [ ]:
!ls gdrive/MyDrive

"'10 CMA student inv.gdoc"
'2023-01-10_16-34-30_hello world.zip'
 4848final.zip
 4848Project.zip
 AbstractLitReview.doc.doc
 AbstractLitReview.doc.gdoc
 Abstract_spring_summary.gdoc
'AllData (1).zip'
 AllData.zip
'ALL THESIS (1).zip'
'ALL THESIS (2).zip'
'ALL THESIS (3).zip'
'ALL THESIS_April_2012_final.zip'
'ALL THESIS.zip'
'APL Interview Schedule.docx'
'Applied Econ.zip'
'AWS Data Scientist Work Sample (008).pdf'
'Bathroom remodel.gdoc'
'bitcoin 11am draft.gdoc'
 bitcoin.gdoc
'CAFB DKDC'
'CALC I Grades.gsheet'
'Colab Notebooks'
'Colorado Venue.gsheet'
 committees.gsheet
'Complex tax code.gdoc'
'Copy of AGI safety fundamentals group exercises.gdoc'
'Copy of Broadcast Analysis 2014-11-15.gdoc'
'Cover Letters'
'DC December Apt Search info.gdoc'
'dec 11 Job application status.gsheet'
'donor assignments - for students - fall 2010.gsheet'
 econ4848.zip
'Econ Paper+4309.zip'
'Euro Econ Backup pre-presentation.zip'
 exchangestudentdataform-sample.gsheet
 ExecutiveSummary.doc.gdoc
'Fall 2011 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install apache_beam mwparserfromhell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp39-cp39-linux_x86_64.whl size=36915 sha256=eb4fdf0eb69526320c3eff1d88d1a8927efa4d3822

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency 

In [ ]:
from datasets import load_dataset

wiki_raw = load_dataset("wikipedia", "20220301.simple")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235M [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
wiki_text_blob = '\n\n'.join(wiki_raw['train']['text'])

In [ ]:
# 200x larger than shakespeare. Great!
len(wiki_text_blob)

215900536

In [ ]:
wiki_text_blob[:1000]

"April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same day of the week as October of the previous year, and in 

In [ ]:
''.join(sorted(list(set(wiki_text_blob))))


'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x80\x81\x82\x83\x85\x88\x89\x8a\x8b\x8f\x92\x96\x9b\x9d\xa0¡¢£¤¥§¨©ª«¬\xad®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġģĤĥĦħĨĩĪīĭįİıĴĵĶķĸĺĻļĽľŁłŃńņňŉŋŌōŏŐőŒœŕŗŘřŚśŜŝŞşŠšŢţťŦŧŨũŪūŬŭůűųŵŷŹźŻżŽžſƁƆƉƊƎƏƒƓƔƚƜƝơƦƩƮưƲƶƷƻƾƿǀǁǂǃǍǎǐǒǔǚǝǣǤǦǧǩǫǭǰǵǶǷǿȘșȚțȜȝȢȦȧȷȸȹɁɂɅɐɑɒɓɔɕɖɗɘəɚɛɜɝɟɠɡɢɣɤɦɧɨɪɫɬɭɯɰɲɳɴɵɶɸɹɺɻɽɾʀʁʂʃʄʇʈʉʊʋʌʍʎʏʐʑʒʔʕʖʘʙʛʜʝʞʟʡʢʣʤʥʦʧʨʩʪʫʬʭʰʱʲʳʷʹʻʼʽʾʿˀˁˆˇˈˉˊˋˌː˚˜˝˞ˡˤ˥˧˨˩˹˺̧̝̞̟̠̣̤̥̩̪̯̱̲̺̼͈̀́̂̃̄̆̇̈̊̌̍̐̓̿͛̚͘͜͡ΆΉΊΌΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩάέήίαβγδεζηθικλμνξοπρςστυφχψωϊόύώϕϘϙϚϛϜϝϟϡϷϸϹϺϻЁЂЃЄЅІЇЈЉЊЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёђѓєѕіїјљњћќўџѠѡѢѣѦѧѪѫѮѰѱѲѳѹҀ҆ҍҎҏҐґҒғҖҘҙҚқҠҡҢңҪҫҮүҰҶҹҺһӀӈӐӑӖӗӘәӣӧӨөӰӲԘԙԱԲԳԴԵԶԷԸԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇՈՉՊՋՌՍՎՏՐՑՒՓՔՕՖաբգդեզէթիլխկհձղճմյնշոչպջռսվտրցւփքօֆևְֲִֵַָֹֻּֿׁׂאבגדהוזחטיךכלםמןנסעףפץצקרשת״،؛؟ءآأؤإئابةتثجحخدذرزسشصضطظعغؿـفقكلمنهوىيًٌٍَُِّْٓٔ١٢٦٧٩ٰٱٲٷٹٽپٿڄڅچڇڈڊڌڑڕژڙښڠڤکڭگڵںڻھہۆۇۈۋ

In [ ]:
##When sorted, low-numbered codes seem to be more "normal"
normal_chars = ''.join(sorted(list(set(wiki_text_blob)))[0:97])
normal_chars


'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [ ]:
test_blob = wiki_text_blob[:10000000]

In [ ]:
test_blob

'April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same day of the week as October of the previous year, and i

In [ ]:
# 30 sec ish
wiki_text_blob_clean = ''.join([x for x in wiki_text_blob if x in normal_chars])

In [ ]:
# Keep 99.7% of chars. great.
len(wiki_text_blob_clean)/len(wiki_text_blob)

0.997293827931951

In [ ]:
encode, decode, vocab_size = get_encoder_decoder_size(wiki_text_blob_clean)

In [ ]:
vocab_size

97

In [ ]:
encode("\n")

[1]

In [ ]:
train, validate = text_to_tv_tensors(wiki_text_blob_clean, encode)

torch.Size([215316272]) torch.int64
tensor([35, 82, 84, 75, 78,  2, 75, 85,  2, 86, 74, 71,  2, 72, 81, 87, 84, 86,
        74,  2, 79, 81, 80, 86, 74,  2, 81, 72,  2, 86, 74, 71,  2, 91, 71, 67,
        84,  2, 75, 80,  2, 86, 74, 71,  2, 44, 87, 78, 75, 67, 80,  2, 67, 80,
        70,  2, 41, 84, 71, 73, 81, 84, 75, 67, 80,  2, 69, 67, 78, 71, 80, 70,
        67, 84, 85, 14,  2, 67, 80, 70,  2, 69, 81, 79, 71, 85,  2, 68, 71, 86,
        89, 71, 71, 80,  2, 47, 67, 84, 69, 74])


In [ ]:
BLOCK_SIZE = 16

train_batch_x, train_batch_y = get_batch(train)
validate_batch_x, validate_batch_y = get_batch(validate)

In [ ]:
seed_raw = "To be or not to "
seed = torch.tensor(encode(seed_raw)).view(1,-1)

m8_wiki = EightGramLanguageModule(vocab_size)
# print(decode(m8.generate(seed, 10)[0,]))
tx, ty = get_batch(train)
_ = m8_wiki(tx)
print(decode(m8_wiki.generate(seed, 20)[0,]))

RuntimeError: ignored

In [ ]:
# optimizer = torch.optim.AdamW(m8_wiki.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(m8_wiki.parameters(), lr=1e-4)
BATCH_SIZE = 128

for epoch in range(10000):
  tx, ty = get_batch(train)

  _, loss = m8_wiki(tx, ty)
  optimizer.zero_grad(set_to_none=True)
  if epoch%1000 == 0:
    print(epoch, loss)
  loss.backward()
  optimizer.step()

RuntimeError: ignored

In [ ]:
print(decode(m8_wiki.generate(seed, 500)[0,]))

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(44)

# See Karpathy 1:40:00 for some suggested hyperparams.

HEAD_SIZE = 16
N_EMB = 32
DROP_RATE = 0.2

class AttentionHead(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(N_EMB, head_size, bias = False)
    self.key = nn.Linear(N_EMB, head_size, bias = False)
    self.value = nn.Linear(N_EMB, head_size, bias = False)

    self.drop = nn.Dropout(DROP_RATE)

    #tril is just a constant, don't want it to "train"
    self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

  def forward(self, inputs):

    B, T, C = inputs.shape
    # print(B, T, C)
    # E.g., 4 x 8 x 63 = examples/batch x chars per example (time) x vocab_size

    q = self.query(inputs) #B x T x head_size
    k = self.key(inputs) #B x T x head_size
    v = self.value(inputs) # B x T x head_size

    wei = q @ torch.transpose(k, 1, 2) * C**-0.5 # B x T X T

    wei = wei.masked_fill(self.tril == 0, float('-inf'))

    wei = torch.softmax(wei, dim=-1) # BxTxT

    wei = self.drop(wei)

    out = wei @ v # B x T x head_size 

    return out


class AttentionMultiHead(nn.Module):
  def __init__(self, num_heads):
    super().__init__()
    head_size = N_EMB//num_heads
    self.heads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_heads)])
    self.lin1 = nn.Linear(N_EMB, N_EMB) #Fuzzy on why I need this... Should always do a linear before an addition?
    self.drop = nn.Dropout(DROP_RATE)

  def forward(self, inputs):
    heads_concat = torch.cat([head(inputs) for head in self.heads], dim=-1)
    heads_lin = self.lin1(heads_concat)
    return self.drop(heads_lin)

class AttentionDecodeBlock(nn.Module):
  def __init__(self):
    super().__init__()
    
    
    self.layer_norm1 = nn.LayerNorm(N_EMB)
    self.multi_head = AttentionMultiHead(4)
    

    self.layer_norm2 = nn.LayerNorm(N_EMB)
    # self.lin1 = nn.Linear(32, 16)
    self.lin1 = nn.Linear(N_EMB, 4*N_EMB) # section 3.3 of AIAYN, bigger "inner" dimension.
    self.relu = nn.ReLU()
    self.lin2 = nn.Linear(4*N_EMB, N_EMB)
    self.drop = nn.Dropout(DROP_RATE)

  def forward(self, inputs):
    
    norm_inputs = self.layer_norm1(inputs) #Karpathy vid suggests deviating from AIAYN: normalize before multi-head and feed forward, not after.
    mhead = self.multi_head(inputs)
    mhead_add = mhead + inputs

    mhead_norm = self.layer_norm2(mhead_add)    
    lin1_out = self.lin1(mhead_norm)
    relu_out = self.relu(lin1_out)
    lin2_out = self.lin2(relu_out) #Fuzzy on why I need this... Should always do a linear before an addition?
    lin2_drop = self.drop(lin2_out)
    lin_add = lin2_out + mhead_add #skip
    lin_norm = lin_add
    # lin_norm = self.layer_norm(lin_add)


    return lin_norm


class AttentionModule(nn.Module):

  def __init__(self):
    super().__init__()
    self.tok_embedding = nn.Embedding(vocab_size, N_EMB)
    self.pos_embedding = nn.Embedding(BLOCK_SIZE, N_EMB)
    self.block1 = AttentionDecodeBlock()
    self.block2 = AttentionDecodeBlock()
    self.block3 = AttentionDecodeBlock()

    self.layer_norm1 = nn.LayerNorm(N_EMB)
    self.lin1 = nn.Linear(N_EMB, vocab_size)

  def forward(self, inputs, targets=None):

    B, T = inputs.shape
    # E.g., 4 x 8 x 63 = examples/batch x chars per example (time) x vocab_size

    tok_emb = self.tok_embedding(inputs)
    pos_emb = self.pos_embedding(torch.arange(BLOCK_SIZE))

    inputs_emb = tok_emb + pos_emb

    # print(embedded_input.shape)

    # fc1 = self.lin1(embedded_input)
    # 4 x 8 x 16

    # fc1_flat = fc1.view(B, -1) #4, 8*16= 128
    # print(fc1_flat.shape)
    # fc2 = self.lin2(fc1_flat) # 4, 65

    b1 = self.block1(inputs_emb)
    b2 = self.block2(b1)
    b3 = self.block3(b2)

    b3_norm = self.layer_norm1(b3)
    logits = self.lin1(b3_norm)

    # It may seem odd that these targets are indices, where logits are scores
    # Basically, pytorch can handle EITHER indices as targets, or the actual OHE/probability ground truths. 
    #   could try target smoothing by OHE to 0.01 and 0.99 instead of 0/1?
    
    if targets is None:
      loss = None
    else:
      # loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
      # print("loss stuff")
      # print(logits.shape)
      # print(targets.shape)
      loss = F.cross_entropy(logits.view(B*T, vocab_size), targets.view(B*T))
      # loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

    return logits, loss

  def generate(self, inputs, max_new_tokens):
    for _ in range(max_new_tokens):
      # print(inputs.shape)
      logits, _ = self(inputs[:,(-1*BLOCK_SIZE):]) #B, T, C
      # print(logits.shape)
      logits_last = logits[:,-1] #end of string only
      probs_last = F.softmax(logits_last, dim=1)
      next_char = torch.multinomial(probs_last, num_samples=1)

      inputs = torch.cat((inputs, next_char), dim=1) #B, T+1

    return inputs



In [ ]:
m_attn = AttentionModule()


In [ ]:

train_batch_x.shape

torch.Size([4, 16])

In [ ]:
m_attn(train_batch_x)[0].shape

torch.Size([4, 16, 97])

In [ ]:
print(decode(m_attn.generate(seed, 100)[0,]))

To be or not to D3@\yDFB).h&[yZVK^HRe#M-wy*3./YG7Dm4	1b#MV*=-_H|({HX~c%G#W@(HA*OF5(vL@q~_E\Oh@@iH|$B={~.j9&\60-g

LB


In [ ]:
# optimizer = torch.optim.AdamW(m8_wiki.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(m_attn.parameters(), lr=1e-3)
BATCH_SIZE = 32

for epoch in range(10000):
  tx, ty = get_batch(train)

  _, loss = m_attn(tx, ty)
  optimizer.zero_grad(set_to_none=True)
  if epoch%1000 == 0:
    print(epoch, loss)
  loss.backward()
  optimizer.step()

0 tensor(2.1195, grad_fn=<NllLossBackward0>)
1000 tensor(2.0128, grad_fn=<NllLossBackward0>)
2000 tensor(2.1310, grad_fn=<NllLossBackward0>)
3000 tensor(2.1260, grad_fn=<NllLossBackward0>)
4000 tensor(1.9886, grad_fn=<NllLossBackward0>)
5000 tensor(1.9867, grad_fn=<NllLossBackward0>)
6000 tensor(1.8632, grad_fn=<NllLossBackward0>)
7000 tensor(1.9447, grad_fn=<NllLossBackward0>)
8000 tensor(2.1570, grad_fn=<NllLossBackward0>)
9000 tensor(2.0334, grad_fn=<NllLossBackward0>)


In [ ]:
eseed_raw = "To be or not to "
seed = torch.tensor(encode(seed_raw)).view(1,-1)

print(decode(m_attn.generate(seed, 200)[0,]))

To be or not to poling Antians, Sup" -- Arria, Amenal. He capital to its lings ins-payle coursmon. 

Reated to of was Agriuct Wedined Sinncows.

Canlanginn Waonjo releathratlecal bage trayes 

A TBatrinal 

Relers, f


Single Multi-head, no lin, no adding, simple_wiki loss = 2.4 

To be or not to ras
Rigamplay chad arich in 102). He to ed in ces fring mongiates- 1956. Olt comak
 1596 arapincalstion. Sprite Ninta MCatacor wagor
 Norvodustelthenickide conka the gomil  Ser than, us of 554)
 Tarte

---------


Above, but with skip connections, and batch norm. Loss = 2.05

To be or not to stilke modeship his come page sauded searanessent Madi be to courd's toSe Autio, IT. This a an Coonrom lithe Sannists 202000)
 meanuzia

Iczy man Livesile Ad'tbaf or a sinces from the Skina
Vaytabo (b

In [ ]:
x = seed[:,(-1*BLOCK_SIZE):]
m_attn(x)[0][:,-1].shape

torch.Size([1, 97])